# Dependencies

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import re
import sys
from datetime import datetime, date
from datetime import timedelta
import pandas as pd

# Get the links of all news articles within a specific range

In [10]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [11]:
dates = []
# Select the start and end date to search on archive
start_date = date(2024,5,1)
end_date = date(2024,5,2)

for single_Date in daterange(start_date, end_date):
    dates.append(single_Date.strftime("%Y-%m-%d"))

options = Options()
options.add_argument("window-size=1920,1080")
service = Service(executable_path="/Users/ashrafulshuvon/Desktop/Selenium/chromedriver")
driver = webdriver.Chrome(service=service, options=options)
links = []

for date in dates:
    url = 'https://www.ittefaq.com.bd/archive/' + str(date)
    driver.get(url)
    time.sleep(5)
    while True:
        try:
            # If the 'show more' button is not visible, then break the loop & extract the links
            mn_button = driver.find_element(By.CSS_SELECTOR, "#ajax_load_more_565_btn")
            if mn_button.get_attribute('style') == "visibility: hidden;":
                print("End of news")
                time.sleep(5)
                link_elements = driver.find_elements(By.CSS_SELECTOR, "a.link_overlay")
                for element in link_elements:
                    links.append(element.get_attribute('href'))
                print("Links extracted")    
                break
            
            else:
                # Click the 'show more' button untill it is not visible/no more newsleft at that range
                more_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#ajax_load_more_565_btn"))
                )

                driver.execute_script("arguments[0].scrollIntoView()", more_button)
                driver.execute_script("arguments[0].click()", more_button)
                print("Clicked")
                time.sleep(5)
            #auto_load_div = driver.find_element(By.CSS_SELECTOR, "#ajax_load_more_565_btn")
            #if auto_load_div.get_attribute('style') == "style: hidden":
            #   print("Visible")

        except ElementClickInterceptedException:
            print("Why?")
driver.quit()

Clicked
Clicked
Clicked
Clicked
End of news
Links extracted


In [12]:
lf = pd.DataFrame(links, columns=['links'])

In [13]:
lf.head(10)

,links
0,https://www.ittefaq.com.bd/685854/%E0%A6%98%E0...
1,https://www.ittefaq.com.bd/685877/%E0%A6%85%E0...
2,https://www.ittefaq.com.bd/685871/%E0%A6%B6%E0...
3,https://www.ittefaq.com.bd/685878/%E0%A6%A1%E0...
4,https://www.ittefaq.com.bd/685876/%E0%A6%97%E0...
5,https://www.ittefaq.com.bd/685875/%E0%A6%85%E0...
6,https://www.ittefaq.com.bd/685874/%E0%A6%AC%E0...
7,https://www.ittefaq.com.bd/685873/%E0%A6%AE%E0...
8,https://www.ittefaq.com.bd/685872/%E0%A6%9C%E0...
9,https://www.ittefaq.com.bd/685870/%E0%A6%B0%E0...


In [14]:
lf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   links   97 non-null     object
dtypes: object(1)
memory usage: 908.0+ bytes


In [15]:
lf.to_csv(f'(Links From {start_date}) TO ({end_date}).csv', index=False)

# Now using the links stored perviously, go to each links and extract the news article, summary, headline

There may be some exception errors because of not finding the elements. That means that, certain article page either has only pictures/videos or the page structure is different than the other ones. In that case article, summararies and headlines for those pages nothin will be saved. To handle those cells with missing values, a code is provided.

In [16]:
service = Service(executable_path="/Users/ashrafulshuvon/Desktop/Selenium/chromedriver")
options = Options()
options.add_argument("window-size=1920,1080")
driver = webdriver.Chrome(service=service, options=options)

cf = pd.read_csv("(Links From 2024-05-01) TO (2024-05-02).csv")

data = []
for url in cf['links']:
    # Open the website
    driver.get(url)

    # Initialize variables to hold content
    summary = ""
    headline = ""
    article = []
    
    # Extract the headline text from the <h1> tag
    try:
        headline_element = driver.find_element(By.CSS_SELECTOR, "div.content_detail_left_inner h1")
        headline = headline_element.text
    except Exception as e:
        print("Error retrieving headline:", e)
        
    # Extract the summary text from the first <p> tag(Taking the first paragraph of the article as summary)
    try:
        summary_element = driver.find_element(By.CSS_SELECTOR, "div.content_detail_content_inner p")
        summary = summary_element.text
    except Exception as e:
        print("Error retrieving summary. Use the headline as summary instead", e)
        summary = headline

    # Extract all paragraph texts within the same div
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.content_detail_content_inner p")
        for paragraph in paragraphs:
            # Add text of each paragraph to the article list, ensure it does not contain newlines
            article.append(paragraph.text.replace('\n', ' '))
    except Exception as e:
        print("Error retrieving article paragraphs:", e)

    # Concatenate all article texts into one string separated by spaces
    article_text = ' '.join(article)

    # Create a DataFrame
    data.append({
        'headline': headline,
        'summary': summary,
        'article': article_text
    })
    
df = pd.DataFrame(data)
time.sleep(5)
# Print and save the DataFrame


# Close the driver
driver.quit()

Error retrieving headline: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.content_detail_left_inner h1"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010112e4a0 chromedriver + 4301984
1   chromedriver                        0x0000000101126de8 chromedriver + 4271592
2   chromedriver                        0x0000000100d5819c chromedriver + 278940
3   chromedriver                        0x0000000100d9a2c4 chromedriver + 549572
4   chromedriver                        0x0000000100dd2c5c chromedriver + 781404
5   chromedriver                        0x0000000100d8f004 chromedriver + 503812
6   chromedriver                        0x0000000100d8f9ec chromedriver + 506348
7   chromedriver                        0x00000001010f64e8 chromedriver + 4072680


KeyboardInterrupt: 

In [ ]:
df

,headline,summary,article
0,ঘুমের যত্ন নেবে যেসব গাছ,সবুজ মনে স্বস্তি এনে দেয়। একঘেয়ে ঘরের সাজসজ্জা...,সবুজ মনে স্বস্তি এনে দেয়। একঘেয়ে ঘরের সাজসজ্জা...
1,অবিলম্বে ১০ম ওয়েজবোর্ড গঠন করুন,মে দিবসের চেতনা বাস্তবায়নে সংবাদপত্র ও টেলিভিশ...,মে দিবসের চেতনা বাস্তবায়নে সংবাদপত্র ও টেলিভিশ...
2,শ্রমিকদের অংশীদারিত্ব নিশ্চিত করাসহ ৪ দফা দাবি,"সংসদ থেকে স্থানীয় সরকার, শিল্প-কারখানা পরিচালন...","সংসদ থেকে স্থানীয় সরকার, শিল্প-কারখানা পরিচালন..."
3,"ডিজাব’র নতুন সভাপতি আলমগীর, সাধারণ সম্পাদক আহম...",সশস্ত্রবাহিনী তথা প্রতিরক্ষা বিটের সাংবাদিকদের...,সশস্ত্রবাহিনী তথা প্রতিরক্ষা বিটের সাংবাদিকদের...
4,গাড়িতে অযাচিত স্টিকার ব্যবহারের বিরুদ্ধে পুলি...,গাড়িতে অযাচিত স্টিকার ব্যবহারের বিরুদ্ধে রাজধ...,গাড়িতে অযাচিত স্টিকার ব্যবহারের বিরুদ্ধে রাজধ...
...,...,...,...
296,সাভারে মাদক ব্যবসায় জড়াচ্ছে নারীরাও,দেশের সীমান্তবর্তী জেলাগুলো থেকে প্রতিদিন বিপু...,দেশের সীমান্তবর্তী জেলাগুলো থেকে প্রতিদিন বিপু...
297,রায়পুরে জনপ্রিয় হচ্ছে সয়াবিন চাষ,রায়পুর উপজেলা জুড়ে আধা পাকা সোনালি সয়াবিনের শি...,রায়পুর উপজেলা জুড়ে আধা পাকা সোনালি সয়াবিনের শি...
298,জোর করে ব্যাংক একীভূতকরণ ঠিক হবে না: ড. ফরাসউদ...,জোর করে ব্যাংক একীভূতকরণ করা ঠিক হবে না বলে মন...,জোর করে ব্যাংক একীভূতকরণ করা ঠিক হবে না বলে মন...
299,"চেয়ারম্যান প্রার্থীকে মারধর সংঘর্ষ, সাংবাদিকদে...",উপজেলা নির্বাচন সামনে রেখে সহিংসতা ছড়িয়ে পড়ছে।...,উপজেলা নির্বাচন সামনে রেখে সহিংসতা ছড়িয়ে পড়ছে।...


Handle the missing values for exceptions

In [ ]:
df = df[~df.apply(lambda x: '' in x.values, axis=1)] # Remove rows with any empty cells


In [ ]:
# Uncomment if start & end dates are not defined. Set the date as per as your link dataframe/dataset
#start_date = date(2024,5,5) 
#end_date = date(2024,5,7)

In [ ]:
df.to_csv(f'News From {start_date} To {end_date}.csv', index=False) # Save the data to a CSV file

In [ ]:
df['headline'].iloc[3]

'ডিজাব’র নতুন সভাপতি আলমগীর, সাধারণ সম্পাদক আহম্মদ উল্লাহ'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 0 to 300
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  297 non-null    object
 1   summary   297 non-null    object
 2   article   297 non-null    object
dtypes: object(3)
memory usage: 9.3+ KB
